In [2]:
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, precision_recall_fscore_support
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [3]:

df = pd.read_csv("C:/Users/kavya/OneDrive/Desktop/Spring2023_890/LoanFHLBData/original/LoanData_preprocessed.csv")
# df=df[['MinPer', 'LTV', 'First', 'BoGender', 'CoAge', 'Rate', 'Self', 'PropType', 'ArmMarg', 'BoEth',"BoRace"]]
df=df[['Year', 'MSA', 'Tract', 'MinPer', 'TraMedY', 'LocMedY', 'CurAreY', 'UPB', 'LTV', 'Purpose', 'FedGuar', 'First', 'CoRace', 'BoGender', 'CoGender', 'CoAge', 'Rate', 'Amount', 'CoCreditScor', 'PMI', 'Self', 'PropType', 'ArmMarg', 'BoEth', 'CoEth','BoRace']]

target = ["BoRace"]
X = df.loc[:, ~df.columns.isin(target)]
y = df.loc[:, df.columns.isin(target)]
feature_names = X.columns

In [4]:

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
scaler = MinMaxScaler(feature_range=(0, 1))

In [5]:

selector = SelectKBest(chi2, k=25)
X_new = selector.fit_transform(X_scaled_df, y)
# print selected features
selected_features = selector.get_support(indices=True)
print([feature_names[i] for i in selected_features])

['Year', 'MSA', 'Tract', 'MinPer', 'TraMedY', 'LocMedY', 'CurAreY', 'UPB', 'LTV', 'Purpose', 'FedGuar', 'First', 'CoRace', 'BoGender', 'CoGender', 'CoAge', 'Rate', 'Amount', 'CoCreditScor', 'PMI', 'Self', 'PropType', 'ArmMarg', 'BoEth', 'CoEth']


In [6]:
# """"""""uncomment for year classification""""
# Define the class labels
# class_labels = {2009: 0, 2010: 1, 2011: 2, 2012: 3, 2013: 4, 2014: 5, 2015: 6, 2016: 7, 2017: 8, 2018: 9, 2019: 10, 2020: 11, 2021: 12}
unique_values = y['BoRace'].unique()
print(unique_values)
class_labels = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5,7: 6}
y[target] = y[target].applymap(class_labels.get)
# Map the year values to class labels
unique_values = y['BoRace'].unique()
print(unique_values)

[5 3 7 2 1 4 6]
[4 2 6 1 0 3 5]


C:\Users\kavya\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
# Print the class distribution before undersampling
print('Before undersampling:')
print(df['BoRace'].value_counts())
from imblearn.under_sampling import RandomUnderSampler

# Perform random undersampling on the majority class
undersampler = RandomUnderSampler(sampling_strategy={4: 10000}, random_state=0)
X_resampled, y_resampled = undersampler.fit_resample(X_scaled_df, y)


# Print the class distribution after undersampling
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)
print('After undersampling:')
print(df_resampled['BoRace'].value_counts())

Before undersampling:
5    661259
7     31146
2     19059
6     16003
3     10976
1      4349
4      1041
Name: BoRace, dtype: int64
After undersampling:
6    31146
1    19059
5    16003
2    10976
4    10000
0     4349
3     1041
Name: BoRace, dtype: int64


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.30, random_state=0)

undersampler = RandomUnderSampler(random_state=42)

xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=13, learning_rate=0.1, max_depth=9, n_estimators=100,
                              colsample_bytree=0.6,subsample=0.3)
# Train the XGBoost model on the training data
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
print("Accuracy:", accuracy)

In [ ]:

from sklearn.metrics import confusion_matrix
from sklearn import metrics

# Assuming y_pred and y_true are the predicted and true labels respectively
cm = confusion_matrix(y_test, y_pred)
# classes=(df['Year'].unique())
classes = (df['BoRace'].unique())

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[1, 2, 3, 4, 5, 6, 7])

cm_display.plot()
plt.show()
# .sort()

In [ ]:

type(cm)
# Print the confusion matrix
print(cm)
classes = [1, 2, 3, 4, 5, 6,7]
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.annotate(str(cm[i][j]), xy=(j, i), ha='center', va='center')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted Label of Borrower race')
plt.ylabel('True Label of Borrower race')
plt.tight_layout()
plt.show()

In [ ]:

importances = xgb_model.feature_importances_
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})

# sort features by importance in descending order
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# plot feature importances
plt.barh(feature_importances['feature'], feature_importances['importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.show()

In [ ]:

similarity_matrix = cm / cm.sum(axis=1, keepdims=True)
plt.figure(figsize=(8, 6))
sns.heatmap(similarity_matrix, annot=True, cmap='Blues')
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.title('Similarity Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:

dx = pd.DataFrame(X_test, columns=X_test.columns)
dx["predicted_value"] = y_pred

corr_matrix = dx.corr()
plt.imshow(corr_matrix, cmap='hot', interpolation='nearest')
plt.xticks(range(len(dx.columns)), dx.columns)
plt.yticks(range(len(dx.columns)), dx.columns)
plt.xticks(rotation=90)
plt.colorbar()
plt.figure(figsize=(15, 10))
plt.subplots_adjust(bottom=0.15)
plt.show()

In [ ]:

# df['FHLBankID'] = fhlbankid_encoder.inverse_transform(df['FHLBankID'])
# df['PropType'] = proptype_encoder.inverse_transform(df['PropType'])
"""Dummy classifier"""
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_pred = dummy_clf.predict(X_test)
dummy_clf.score(y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
print("Accuracy:", accuracy)



